In [125]:
import pandas as pd
from fredapi import Fred
import sys
import os
import hvplot.pandas as hvplot
# Add config module to sys path
sys.path.insert(0, os.path.abspath('../../config'))

# Import API key
from api_keys import fred_api_key


In [126]:
#Set up API Key

fred = Fred(api_key=f'{fred_api_key}')

In [127]:
#Retrieve 10 Year Treasury security rates (monthly) from FRED API

data_10year_treasury = fred.get_series('GS10')

In [128]:
# Retain only data since June 1976. This is to match with data available for 2 year Treasury securities
data_10year_treasury = data_10year_treasury.loc[data_10year_treasury.index > '1958-12-31']

In [129]:
data_10year_treasury.head(5)

1959-01-01    4.02
1959-02-01    3.96
1959-03-01    3.99
1959-04-01    4.12
1959-05-01    4.31
dtype: float64

In [130]:
data_2year_treasury = fred.get_series('GS2')

In [131]:
data_2year_treasury.head()

1976-06-01    7.06
1976-07-01    6.85
1976-08-01    6.63
1976-09-01    6.42
1976-10-01    5.98
dtype: float64

In [132]:
data_1year_treasury = fred.get_series('GS1')

In [133]:
data_1year_treasury = data_1year_treasury.loc[data_1year_treasury.index > '1958-12-31']

In [134]:
data_1year_treasury.head()

1959-01-01    3.36
1959-02-01    3.54
1959-03-01    3.61
1959-04-01    3.72
1959-05-01    3.96
dtype: float64

In [135]:
data_10year_treasury_df = pd.DataFrame(data_10year_treasury)

data_2year_treasury_df = pd.DataFrame(data_2year_treasury)

data_1year_treasury_df = pd.DataFrame(data_1year_treasury)


In [136]:
data_10year_treasury_df = data_10year_treasury_df.reset_index()

In [137]:
data_10year_treasury_df.head()

,index,0
0,1959-01-01,4.02
1,1959-02-01,3.96
2,1959-03-01,3.99
3,1959-04-01,4.12
4,1959-05-01,4.31


In [138]:
data_2year_treasury_df = data_2year_treasury_df.reset_index()

In [139]:
data_2year_treasury_df.head()

,index,0
0,1976-06-01,7.06
1,1976-07-01,6.85
2,1976-08-01,6.63
3,1976-09-01,6.42
4,1976-10-01,5.98


In [140]:
data_1year_treasury_df = data_1year_treasury_df.reset_index()

In [141]:
data_1year_treasury_df.head()

,index,0
0,1959-01-01,3.36
1,1959-02-01,3.54
2,1959-03-01,3.61
3,1959-04-01,3.72
4,1959-05-01,3.96


In [152]:
treasury_df = pd.merge(data_10year_treasury_df,data_2year_treasury_df, on="index", how="left")

In [155]:
treasury_df = treasury_df.fillna(0)

In [156]:
treasury_df.head()

,index,0_x,0_y
0,1959-01-01,4.02,0.0
1,1959-02-01,3.96,0.0
2,1959-03-01,3.99,0.0
3,1959-04-01,4.12,0.0
4,1959-05-01,4.31,0.0


In [157]:
treasury_df = pd.merge(treasury_df, data_1year_treasury_df, on="index")

In [158]:
treasury_df.head()

,index,0_x,0_y,0
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [159]:
treasury_df = treasury_df.rename(columns={"index":"date", 
                                          "0_x":"10 Years T Rates", 
                                          "0_y": "2 Years T Rates",
                                         0 : "1 Year T Rates"})

In [160]:
treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [169]:
def adjust_2_year_missing_rates(twoYearValue, tenYearValue):
    if twoYearValue == 0:
        return tenYearValue
    else:
        return twoYearValue

In [175]:
treasury_df["2 Years T Rates"] = treasury_df.apply(lambda row :  adjust_2_year_missing_rates(row["2 Years T Rates"], row["10 Years T Rates"]), axis=1)

In [176]:
treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates,10-2 Rate Diff,10-1 Rate Diff
0,1959-01-01,4.02,4.02,3.36,4.02,0.66
1,1959-02-01,3.96,3.96,3.54,3.96,0.42
2,1959-03-01,3.99,3.99,3.61,3.99,0.38
3,1959-04-01,4.12,4.12,3.72,4.12,0.40
4,1959-05-01,4.31,4.31,3.96,4.31,0.35


In [177]:
treasury_df["10-2 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["2 Years T Rates"]

In [178]:
treasury_df["10-1 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["1 Year T Rates"]

In [179]:
treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates,10-2 Rate Diff,10-1 Rate Diff
0,1959-01-01,4.02,4.02,3.36,0.0,0.66
1,1959-02-01,3.96,3.96,3.54,0.0,0.42
2,1959-03-01,3.99,3.99,3.61,0.0,0.38
3,1959-04-01,4.12,4.12,3.72,0.0,0.40
4,1959-05-01,4.31,4.31,3.96,0.0,0.35


In [180]:
treasury_df.hvplot.line(x="date", y="10-2 Rate Diff")

:Curve   [date]   (10-2 Rate Diff)

In [181]:
treasury_df.hvplot.line(x="date", y="10-1 Rate Diff")

:Curve   [date]   (10-1 Rate Diff)